# 文本相似度实例

## Step1 导入相关包

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset

/home/pika/App/miniconda3/envs/transformers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [2]:
dataset = load_dataset("json", data_files="./train_pair_1w.json", split="train")
dataset

Generating train split: 10000 examples [00:00, 1541796.79 examples/s]


Dataset({
    features: ['sentence1', 'sentence2', 'label'],
    num_rows: 10000
})

In [3]:
dataset[0]

{'sentence1': '找一部小时候的动画片', 'sentence2': '求一部小时候的动画片。谢了', 'label': '1'}

## Step3 划分数据集

In [4]:
datasets = dataset.train_test_split(test_size=0.2)
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
})

In [5]:
datasets["train"][0]

{'sentence1': '我手下有一个很有机谋的人，极得我的信任，是我派他去监视南方的动静的。',
 'sentence2': '我派了我手下一个很有头脑的人去南方视察了一下动态。',
 'label': '1'}

## Step4 数据集预处理

In [6]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")


def process_function(examples):
    tokenized_examples = tokenizer(
        examples["sentence1"], examples["sentence2"], max_length=128, truncation=True
    )
    # 均方误差做loss，所以这里要float
    tokenized_examples["labels"] = [float(label) for label in examples["label"]]
    return tokenized_examples


tokenized_datasets = datasets.map(
    process_function, batched=True, remove_columns=datasets["train"].column_names
)
tokenized_datasets

Map: 100%|██████████| 2000/2000 [00:00<00:00, 31511.83 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [7]:
print(tokenized_datasets["train"][0])

{'input_ids': [101, 2769, 2797, 678, 3300, 671, 702, 2523, 3300, 3322, 6450, 4638, 782, 8024, 3353, 2533, 2769, 4638, 928, 818, 8024, 3221, 2769, 3836, 800, 1343, 4664, 6228, 1298, 3175, 4638, 1220, 7474, 4638, 511, 102, 2769, 3836, 749, 2769, 2797, 678, 671, 702, 2523, 3300, 1928, 5554, 4638, 782, 1343, 1298, 3175, 6228, 2175, 749, 671, 678, 1220, 2578, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': 1.0}


## Step5 创建模型

In [8]:
from transformers import BertForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "hfl/chinese-macbert-base", num_labels=1
)  # num_labels=1，回归任务，损失用MSE

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step6 创建评估函数

In [9]:
import evaluate

acc_metric = evaluate.load("./metric_accuracy.py")
f1_metirc = evaluate.load("./metric_f1.py")

In [10]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    ########################33
    predictions = [int(p > 0.5) for p in predictions]
    ##############################
    labels = [int(l) for l in labels]
    # predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metirc.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

## Step7 创建TrainingArguments

In [11]:
train_args = TrainingArguments(
    output_dir="./cross_model",  # 输出文件夹
    per_device_train_batch_size=32,  # 训练时的batch_size
    per_device_eval_batch_size=32,  # 验证时的batch_size
    logging_steps=10,  # log 打印的频率
    eval_strategy="epoch",  # 评估策略
    save_strategy="epoch",  # 保存策略
    save_total_limit=3,  # 最大保存数
    learning_rate=2e-5,  # 学习率
    weight_decay=0.01,  # weight_decay
    metric_for_best_model="f1",  # 设定评估指标
    load_best_model_at_end=True,
)  # 训练完成后加载最优模型
train_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=epoch,
evaluation_strategy=None,
fp16=False,
fp16_backend=auto,
fp

## Step8 创建Trainer

In [12]:
from transformers import DataCollatorWithPadding

trainer = Trainer(
    model=model,
    args=train_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=eval_metric,
)

## Step9 模型训练

In [13]:
trainer.train()

  1%|▏         | 10/750 [00:03<03:33,  3.46it/s]

{'loss': 0.6207, 'grad_norm': 9.281704902648926, 'learning_rate': 1.9733333333333336e-05, 'epoch': 0.04}


  3%|▎         | 20/750 [00:05<03:17,  3.71it/s]

{'loss': 0.1761, 'grad_norm': 4.04828405380249, 'learning_rate': 1.9466666666666668e-05, 'epoch': 0.08}


  4%|▍         | 31/750 [00:08<02:58,  4.03it/s]

...

/tmp/ipykernel_23242/2054007762.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions = [int(p > 0.5) for p in predictions]
                                                 
100%|██████████| 750/750 [03:40<00:00,  3.80it/s]

{'eval_loss': 0.07383160293102264, 'eval_accuracy': 0.8995, 'eval_f1': 0.8683693516699411, 'eval_runtime': 4.7314, 'eval_samples_per_second': 422.707, 'eval_steps_per_second': 13.315, 'epoch': 3.0}


100%|██████████| 750/750 [03:42<00:00,  3.38it/s]

{'train_runtime': 222.0941, 'train_samples_per_second': 108.062, 'train_steps_per_second': 3.377, 'train_loss': 0.09784200775623321, 'epoch': 3.0}


TrainOutput(global_step=750, training_loss=0.09784200775623321, metrics={'train_runtime': 222.0941, 'train_samples_per_second': 108.062, 'train_steps_per_second': 3.377, 'total_flos': 1556485250567424.0, 'train_loss': 0.09784200775623321, 'epoch': 3.0})

## Step10 模型评估

In [14]:
trainer.evaluate(tokenized_datasets["test"])

100%|██████████| 63/63 [00:05<00:00, 12.87it/s]/tmp/ipykernel_23242/2054007762.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions = [int(p > 0.5) for p in predictions]
100%|██████████| 63/63 [00:05<00:00, 12.08it/s]


{'eval_loss': 0.07383160293102264,
 'eval_accuracy': 0.8995,
 'eval_f1': 0.8683693516699411,
 'eval_runtime': 5.3037,
 'eval_samples_per_second': 377.093,
 'eval_steps_per_second': 11.878,
 'epoch': 3.0}

## Step11 模型预测

In [15]:
from transformers import pipeline, TextClassificationPipeline

In [16]:
model.config.id2label = {0: "不相似", 1: "相似"}

In [17]:
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [18]:
pipe({"text": "我喜欢北京", "text_pair": "天气怎样"})

{'label': '不相似', 'score': 1.0}

In [19]:
pipe({"text": "我喜欢北京", "text_pair": "天气怎样"}, function_to_apply="none")

{'label': '不相似', 'score': 0.1161593645811081}

In [20]:
result = pipe({"text": "我喜欢北京", "text_pair": "天气怎样"}, function_to_apply="none")
result["label"] = "相似" if result["score"] > 0.5 else "不相似"
result

{'label': '不相似', 'score': 0.1161593645811081}